# smooth data and fit first level GLM

K. Garner, 2022  
This code will smooth the data and fit a first level GLM to each participant's fmri data.  
Assumptions:  
Data is BIDS enough.  


## set up environment

In [1]:
from bids.layout import BIDSLayout
#from nipype.interfaces import afni 
from nipype.interfaces.io import BIDSDataGrabber, DataFinder, DataSink, DataGrabber
import nipype.pipeline as pe
import nipype as ni
from nipype.interfaces.utility import Function
import nipype.interfaces.fsl.maths as fsl
from nipype.interfaces import spm as spm
from nipype.algorithms import modelgen as mgen
from nipype.algorithms.misc import Gunzip 
import pandas as pd
import os, re, json
# https://nipype.readthedocs.io/en/0.11.0/users/spmmcr.html

matlab_cmd = '/opt/spm12-r7219/run_spm12.sh /opt/matlabmcr-2010a/v713/ script'
spm.SPMCommand.set_mlab_paths(matlab_cmd=matlab_cmd, use_mcr=True)

220419-12:39:20,272 nipype.utils WARNING:
	 A newer version (1.7.1) of nipy/nipype is available. You are using 1.5.0


## define source data location and establish workflow

In [2]:
basedir = '/scratch/qbi/uqkgarn1/data/'
layout = BIDSLayout(basedir)
subs = layout.get_subjects()

glm = pe.Workflow(name='glms') # workflow to run the analysis

/opt/miniconda-latest/envs/neuro/lib/python3.6/site-packages/bids/layout/models.py:102: FutureWarning: The 'extension' entity currently excludes the leading dot ('.'). As of version 0.14.0, it will include the leading dot. To suppress this warning and include the leading dot, use `bids.config.set_option('extension_initial_dot', True)`.
  FutureWarning)


## data grabbing

In [3]:
dg = pe.Node(DataGrabber(infields= ['sub'], 
                         outfields=['func', 'motion', 'onsets', 'bjson', 'mask']), 
                         name='data-grabber')
dg.inputs.base_dir = '/scratch/qbi/uqkgarn1/data/derivatives/fmriprep/'


dg.inputs.sort_filelist = True
dg.inputs.template='*'
dg.inputs.template_args = {'func': [['sub', 'sub']],
                           'motion':[['sub', 'sub']],
                           'onsets':[['sub', 'sub']],
                           'bjson':[['sub', 'sub']],
                           'mask':[['sub', 'sub']]}
dg.inputs.field_template = {'func':   '/scratch/qbi/uqkgarn1/data/derivatives/fmriprep/sub-%s/ses-02/func/sub-%s_*_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz',
                            'motion': '/scratch/qbi/uqkgarn1/data/derivatives/fmriprep/sub-%s/ses-02/func/sub-%s_ses-02_task-attlearn_run-*_desc-motion-physregress_timeseries.txt',
                            'onsets': '/scratch/qbi/uqkgarn1/data/derivatives/fmriprep/sub-%s/ses-02/beh/sub-%s_ses-02_task-attlearn_run-*_desc-glm-onsets.json',
                            'bjson':  '/scratch/qbi/uqkgarn1/data/derivatives/fmriprep/sub-%s/ses-02/func/sub-%s_ses-02_task-attlearn_run-*_space-MNI152NLin2009cAsym_desc-preproc_bold.json',
                            'mask':   '/scratch/qbi/uqkgarn1/data/derivatives/fmriprep/sub-%s/ses-02/anat/sub-%s_ses-02_space-MNI152NLin2009cAsym_label-GM_probseg.nii.gz'}

In [4]:
dg.inputs.sub = subs[0]
res = dg.run()
res.outputs

220419-12:39:22,721 nipype.workflow INFO:
	 [Node] Setting-up "data-grabber" in "/tmp/tmpu8wufy0z/data-grabber".
220419-12:39:22,727 nipype.workflow INFO:
	 [Node] Running "data-grabber" ("nipype.interfaces.io.DataGrabber")
220419-12:39:22,735 nipype.workflow INFO:
	 [Node] Finished "data-grabber".



bjson = ['/scratch/qbi/uqkgarn1/data/derivatives/fmriprep/sub-01/ses-02/func/sub-01_ses-02_task-attlearn_run-1_space-MNI152NLin2009cAsym_desc-preproc_bold.json', '/scratch/qbi/uqkgarn1/data/derivatives/fmriprep/sub-01/ses-02/func/sub-01_ses-02_task-attlearn_run-2_space-MNI152NLin2009cAsym_desc-preproc_bold.json', '/scratch/qbi/uqkgarn1/data/derivatives/fmriprep/sub-01/ses-02/func/sub-01_ses-02_task-attlearn_run-3_space-MNI152NLin2009cAsym_desc-preproc_bold.json']
func = ['/scratch/qbi/uqkgarn1/data/derivatives/fmriprep/sub-01/ses-02/func/sub-01_ses-02_task-attlearn_run-1_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz', '/scratch/qbi/uqkgarn1/data/derivatives/fmriprep/sub-01/ses-02/func/sub-01_ses-02_task-attlearn_run-2_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz', '/scratch/qbi/uqkgarn1/data/derivatives/fmriprep/sub-01/ses-02/func/sub-01_ses-02_task-attlearn_run-3_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz']
mask = /scratch/qbi/uqkgarn1/data/derivatives/fmriprep/sub

In [5]:
info = pe.Node(ni.IdentityInterface(fields=['sub', 'sub']), name = 'info')
info.iterables = [('sub', ['01'])]

In [6]:
def printSubPath(full_file_path):
    # function to split filepath into constituent parts, then print string to add as input to DataSink for the 
    # container string
    # given the full filepath, this extracts the subj/ses folder names for input
    # into DataSink
    # Args
    # -- full_file_path: cell of file names, any one of the data grabber outputs should work
    # Returns
    # -- string of the form 'sub'0x/ses-0x'
    import os
    import re
    fname = os.path.normpath(full_file_path[0])
    l = fname.split(os.sep)
    sub = [s for s in l if re.search('sub', s)][0]
    ses = [s for s in l if re.search('ses', s)][0]
    name = [sub, ses]
    name = '/'.join(name)
    return name



## datasink node for data saves

In [7]:
ds = pe.Node(DataSink(), name='sink-stuff')
ds.inputs.base_directory = "/scratch/qbi/uqkgarn1/data/derivatives/spm/"
substitutions = [('_sub_([0-9]*)', '')]
ds.inputs.regexp_substitutions = substitutions

## get design info

In [8]:
def getOnsetsJson(input_files):
    # reads the onsets for GLM from each run, and puts together as a bunch, ready for use in spm/model def
    # Args:
    # -- input_files [cell list of filenames] output as 'onsets' from datagrabber
    # Returns
    # a Bunch of dim n runs, containing the fields 'conditions', 'onsets', 'durations'
    from nipype.interfaces.base import Bunch
    import json
    prt_output = [] #prt=protocol
    count = 0
    for f in input_files: 
        count = count + 1
        with open(f, "r+") as file:
            data = json.load(file)
            prt_output.insert(count, 
                              Bunch(conditions=data['names'],
                                    onsets=data['onsets'],
                                    durations=data['durations']))
    return prt_output

In [9]:
# now a node to use the getOnsetsJson function in the work flow
get_onsets = pe.Node(Function(input_names = ['input_files'],
                              output_names = ['prt_output'],
                              function = getOnsetsJson),
                     name = 'get_prt_onsets')

## some unzipping modules so that spm can handle the images

Note: I opted to create 2 to avoid output conflicts

In [10]:
gunzip_func = pe.MapNode(Gunzip(), name='gunzipfunc', iterfield=['in_file'])
gunzip_mask = pe.Node(Gunzip(), name='gunzipmask')

## smooth the functional data

In [11]:
smooth = pe.Node(spm.Smooth(), name='smooth')
smooth.inputs.fwhm = [2, 2, 2]  # play with this for sub 1 

## grab the TR for the model specification

In [12]:
def getTRJson(input_files):
    # extracts the TR in seconds from one of the json files accompanying the T2 images
    # Args
    # -- input_files: cell list of json files for bold images (bjson from data grabber)
    # Returns
    # -- TR [numeric]: TR is seconds
    import json
    with open(input_files[0], "r+") as file:
            data = json.load(file)
            TR = data['RepetitionTime'] 
    return TR

In [13]:
get_tr = pe.Node(Function(input_names=['input_files'],
                          output_names=['TR'],
                          function=getTRJson),
                 name='get_TR')

## specify glm, generate design matrix and estimate

In [14]:
model_spec = pe.Node(mgen.SpecifySPMModel(concatenate_runs=False,
                                         input_units='secs',
                                         output_units='secs',
                                         high_pass_filter_cutoff=128),
                     name="modelspec")

In [15]:
level_1_design = pe.Node(spm.Level1Design(bases={'hrf': {'derivs': [1, 1]}},
                                 timing_units='secs',
                                 model_serial_correlations='FAST'),
                         name="level1design")

In [16]:
estimate = pe.Node(spm.EstimateModel(estimation_method={'Classical': 1},
                                     write_residuals=False),
                                     name="estimate")

## Connect workflow and run

In [17]:
glm.connect([(info, dg, [('sub', 'sub')]), # get sub info to feed into data grabber
             (dg, ds, [(('motion', printSubPath), # print the path for saving data to (i.e. sub-0x/ses-0x)
                         'container')]),
             (dg, get_onsets, [('onsets', 'input_files')]), # get the onsets for each run into a single set of bunches
             (dg, gunzip_func, [('func', 'in_file')]), # unzip t2
             (dg, gunzip_mask, [('mask', 'in_file')]), # unzip masks
             (gunzip_func, smooth, [('out_file', 'in_files')]), # smooth data
             (smooth, ds, [('smoothed_files', 'data.@smooth')]), # save smoothed
             (dg, get_tr, [('bjson', 'input_files')]), # get TR for model definition
             (get_tr, model_spec, [('TR', 'time_repetition')]), # TR for model spec
             (dg, model_spec, [('motion', 'realignment_parameters')]),
             (get_onsets, model_spec, [('prt_output', 'subject_info')]),
             (smooth, model_spec, [('smoothed_files', 'functional_runs')]),
             (get_tr, level_1_design, [('TR', 'interscan_interval')]),
             (model_spec, level_1_design, [('session_info', 'session_info')]),
             (gunzip_mask, level_1_design, [('out_file', 'mask_image')]), 
             (level_1_design, estimate, [('spm_mat_file', 'spm_mat_file')]),
             (estimate, ds, [('beta_images', 'flglm.@beta')]),
             (estimate, ds, [('spm_mat_file', 'flglm.@des')]),
             (estimate, ds, [('residual_image', 'flglm.@res')]),
             (estimate, ds, [('mask_image', 'flglm.@mask')])
])
glm.run()          

220419-12:39:22,858 nipype.workflow INFO:
	 Workflow glms settings: ['check', 'execution', 'logging', 'monitoring']
220419-12:39:22,875 nipype.workflow INFO:
	 Running serially.
220419-12:39:22,875 nipype.workflow INFO:
	 [Node] Setting-up "glms.data-grabber" in "/tmp/tmpu8wufy0z/data-grabber".
220419-12:39:22,877 nipype.workflow INFO:
	 [Node] Outdated cache found for "glms.data-grabber".
220419-12:39:22,881 nipype.workflow INFO:
	 [Node] Running "data-grabber" ("nipype.interfaces.io.DataGrabber")
220419-12:39:22,886 nipype.workflow INFO:
	 [Node] Finished "glms.data-grabber".
220419-12:39:22,887 nipype.workflow INFO:
	 [Node] Setting-up "glms.get_TR" in "/tmp/tmpexgtspwo/glms/_sub_01/get_TR".
220419-12:39:22,892 nipype.workflow INFO:
	 [Node] Running "get_TR" ("nipype.interfaces.utility.wrappers.Function")
220419-12:39:22,895 nipype.workflow INFO:
	 [Node] Finished "glms.get_TR".
220419-12:39:22,896 nipype.workflow INFO:
	 [Node] Setting-up "glms.gunzipmask" in "/tmp/tmpm7nzddvw/glms

220419-13:47:44,973 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/data/derivatives/spm/sub-01/ses-02/flglm/_sub_01/beta_0015.nii -> /scratch/qbi/uqkgarn1/data/derivatives/spm/sub-01/ses-02/flglm//beta_0015.nii
220419-13:47:45,18 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/data/derivatives/spm/sub-01/ses-02/flglm/_sub_01/beta_0016.nii -> /scratch/qbi/uqkgarn1/data/derivatives/spm/sub-01/ses-02/flglm//beta_0016.nii
220419-13:47:45,25 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/data/derivatives/spm/sub-01/ses-02/flglm/_sub_01/beta_0017.nii -> /scratch/qbi/uqkgarn1/data/derivatives/spm/sub-01/ses-02/flglm//beta_0017.nii
220419-13:47:45,36 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/data/derivatives/spm/sub-01/ses-02/flglm/_sub_01/beta_0018.nii -> /scratch/qbi/uqkgarn1/data/derivatives/spm/sub-01/ses-02/flglm//beta_0018.nii
220419-13:47:45,43 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/data/derivatives/spm/sub-01/ses-02/flglm/_sub_01/beta_0019.nii -> /s

220419-13:47:45,845 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/data/derivatives/spm/sub-01/ses-02/flglm/_sub_01/beta_0053.nii -> /scratch/qbi/uqkgarn1/data/derivatives/spm/sub-01/ses-02/flglm//beta_0053.nii
220419-13:47:45,852 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/data/derivatives/spm/sub-01/ses-02/flglm/_sub_01/beta_0054.nii -> /scratch/qbi/uqkgarn1/data/derivatives/spm/sub-01/ses-02/flglm//beta_0054.nii
220419-13:47:45,892 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/data/derivatives/spm/sub-01/ses-02/flglm/_sub_01/beta_0055.nii -> /scratch/qbi/uqkgarn1/data/derivatives/spm/sub-01/ses-02/flglm//beta_0055.nii
220419-13:47:45,899 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/data/derivatives/spm/sub-01/ses-02/flglm/_sub_01/beta_0056.nii -> /scratch/qbi/uqkgarn1/data/derivatives/spm/sub-01/ses-02/flglm//beta_0056.nii
220419-13:47:45,982 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/data/derivatives/spm/sub-01/ses-02/flglm/_sub_01/beta_0057.nii -

220419-13:47:46,880 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/data/derivatives/spm/sub-01/ses-02/flglm/_sub_01/beta_0091.nii -> /scratch/qbi/uqkgarn1/data/derivatives/spm/sub-01/ses-02/flglm//beta_0091.nii
220419-13:47:46,887 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/data/derivatives/spm/sub-01/ses-02/flglm/_sub_01/beta_0092.nii -> /scratch/qbi/uqkgarn1/data/derivatives/spm/sub-01/ses-02/flglm//beta_0092.nii
220419-13:47:46,996 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/data/derivatives/spm/sub-01/ses-02/flglm/_sub_01/beta_0093.nii -> /scratch/qbi/uqkgarn1/data/derivatives/spm/sub-01/ses-02/flglm//beta_0093.nii
220419-13:47:47,3 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/data/derivatives/spm/sub-01/ses-02/flglm/_sub_01/beta_0094.nii -> /scratch/qbi/uqkgarn1/data/derivatives/spm/sub-01/ses-02/flglm//beta_0094.nii
220419-13:47:47,50 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/data/derivatives/spm/sub-01/ses-02/flglm/_sub_01/beta_0095.nii -> /

220419-13:47:48,85 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/data/derivatives/spm/sub-01/ses-02/flglm/_sub_01/beta_0129.nii -> /scratch/qbi/uqkgarn1/data/derivatives/spm/sub-01/ses-02/flglm//beta_0129.nii
220419-13:47:48,93 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/data/derivatives/spm/sub-01/ses-02/flglm/_sub_01/beta_0130.nii -> /scratch/qbi/uqkgarn1/data/derivatives/spm/sub-01/ses-02/flglm//beta_0130.nii
220419-13:47:48,132 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/data/derivatives/spm/sub-01/ses-02/flglm/_sub_01/beta_0131.nii -> /scratch/qbi/uqkgarn1/data/derivatives/spm/sub-01/ses-02/flglm//beta_0131.nii
220419-13:47:48,139 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/data/derivatives/spm/sub-01/ses-02/flglm/_sub_01/beta_0132.nii -> /scratch/qbi/uqkgarn1/data/derivatives/spm/sub-01/ses-02/flglm//beta_0132.nii
220419-13:47:48,188 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/data/derivatives/spm/sub-01/ses-02/flglm/_sub_01/beta_0133.nii -> 

...

220419-13:48:22,449 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/data/derivatives/spm/sub-01/ses-02/flglm/_sub_01/beta_1231.nii -> /scratch/qbi/uqkgarn1/data/derivatives/spm/sub-01/ses-02/flglm//beta_1231.nii
220419-13:48:22,456 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/data/derivatives/spm/sub-01/ses-02/flglm/_sub_01/beta_1232.nii -> /scratch/qbi/uqkgarn1/data/derivatives/spm/sub-01/ses-02/flglm//beta_1232.nii
220419-13:48:22,493 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/data/derivatives/spm/sub-01/ses-02/flglm/_sub_01/beta_1233.nii -> /scratch/qbi/uqkgarn1/data/derivatives/spm/sub-01/ses-02/flglm//beta_1233.nii
220419-13:48:22,542 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/data/derivatives/spm/sub-01/ses-02/flglm/_sub_01/beta_1234.nii -> /scratch/qbi/uqkgarn1/data/derivatives/spm/sub-01/ses-02/flglm//beta_1234.nii
220419-13:48:22,549 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/data/derivatives/spm/sub-01/ses-02/flglm/_sub_01/beta_1235.nii -

220419-13:48:23,707 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/data/derivatives/spm/sub-01/ses-02/flglm/_sub_01/beta_1269.nii -> /scratch/qbi/uqkgarn1/data/derivatives/spm/sub-01/ses-02/flglm//beta_1269.nii
220419-13:48:23,714 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/data/derivatives/spm/sub-01/ses-02/flglm/_sub_01/beta_1270.nii -> /scratch/qbi/uqkgarn1/data/derivatives/spm/sub-01/ses-02/flglm//beta_1270.nii
220419-13:48:23,747 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/data/derivatives/spm/sub-01/ses-02/flglm/_sub_01/beta_1271.nii -> /scratch/qbi/uqkgarn1/data/derivatives/spm/sub-01/ses-02/flglm//beta_1271.nii
220419-13:48:23,785 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/data/derivatives/spm/sub-01/ses-02/flglm/_sub_01/beta_1272.nii -> /scratch/qbi/uqkgarn1/data/derivatives/spm/sub-01/ses-02/flglm//beta_1272.nii
220419-13:48:23,821 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/data/derivatives/spm/sub-01/ses-02/flglm/_sub_01/beta_1273.nii -

220419-13:48:24,898 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/data/derivatives/spm/sub-01/ses-02/flglm/_sub_01/beta_1307.nii -> /scratch/qbi/uqkgarn1/data/derivatives/spm/sub-01/ses-02/flglm//beta_1307.nii
220419-13:48:24,905 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/data/derivatives/spm/sub-01/ses-02/flglm/_sub_01/beta_1308.nii -> /scratch/qbi/uqkgarn1/data/derivatives/spm/sub-01/ses-02/flglm//beta_1308.nii
220419-13:48:24,913 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/data/derivatives/spm/sub-01/ses-02/flglm/_sub_01/beta_1309.nii -> /scratch/qbi/uqkgarn1/data/derivatives/spm/sub-01/ses-02/flglm//beta_1309.nii
220419-13:48:24,948 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/data/derivatives/spm/sub-01/ses-02/flglm/_sub_01/beta_1310.nii -> /scratch/qbi/uqkgarn1/data/derivatives/spm/sub-01/ses-02/flglm//beta_1310.nii
220419-13:48:24,985 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/data/derivatives/spm/sub-01/ses-02/flglm/_sub_01/beta_1311.nii -

220419-13:48:26,12 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/data/derivatives/spm/sub-01/ses-02/flglm/_sub_01/beta_1345.nii -> /scratch/qbi/uqkgarn1/data/derivatives/spm/sub-01/ses-02/flglm//beta_1345.nii
220419-13:48:26,56 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/data/derivatives/spm/sub-01/ses-02/flglm/_sub_01/beta_1346.nii -> /scratch/qbi/uqkgarn1/data/derivatives/spm/sub-01/ses-02/flglm//beta_1346.nii
220419-13:48:26,95 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/data/derivatives/spm/sub-01/ses-02/flglm/_sub_01/beta_1347.nii -> /scratch/qbi/uqkgarn1/data/derivatives/spm/sub-01/ses-02/flglm//beta_1347.nii
220419-13:48:26,162 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/data/derivatives/spm/sub-01/ses-02/flglm/_sub_01/beta_1348.nii -> /scratch/qbi/uqkgarn1/data/derivatives/spm/sub-01/ses-02/flglm//beta_1348.nii
220419-13:48:26,228 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/data/derivatives/spm/sub-01/ses-02/flglm/_sub_01/beta_1349.nii -> /

220419-13:48:27,306 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/data/derivatives/spm/sub-01/ses-02/flglm/_sub_01/beta_1383.nii -> /scratch/qbi/uqkgarn1/data/derivatives/spm/sub-01/ses-02/flglm//beta_1383.nii
220419-13:48:27,344 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/data/derivatives/spm/sub-01/ses-02/flglm/_sub_01/beta_1384.nii -> /scratch/qbi/uqkgarn1/data/derivatives/spm/sub-01/ses-02/flglm//beta_1384.nii
220419-13:48:27,386 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/data/derivatives/spm/sub-01/ses-02/flglm/_sub_01/beta_1385.nii -> /scratch/qbi/uqkgarn1/data/derivatives/spm/sub-01/ses-02/flglm//beta_1385.nii
220419-13:48:27,394 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/data/derivatives/spm/sub-01/ses-02/flglm/_sub_01/beta_1386.nii -> /scratch/qbi/uqkgarn1/data/derivatives/spm/sub-01/ses-02/flglm//beta_1386.nii
220419-13:48:27,442 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/data/derivatives/spm/sub-01/ses-02/flglm/_sub_01/beta_1387.nii -